In [9]:
# !pip install kaggle

In [10]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("suchintikasarkar/sentiment-analysis-for-mental-health")

# print("Path to dataset files:", path)

In [ ]:
import os
print(os.listdir())

['bert_finetuned', 'BERT_Sentiment analysis.ipynb', 'Chatbot.ipynb', 'dataset', 'Naive Bayes_Sentimental analysis.ipynb', 'results']


Setting tools, library and download dataset

In [12]:
import re

import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import math

import nltk

from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict

import dataset



df = pd.read_csv("dataset\Sentiment Analysis for Mental Health.csv")
df = df.sample(frac=0.1, random_state=42)

df

,Unnamed: 0,statement,status
22261,22261,Just as the the title says. I feel like one is...,Depression
41400,41400,a blackened sky encroached tugging behind it m...,Depression
20065,20065,"It gives you insomnia, which in turn makes you...",Depression
30036,30036,"Hello all, I'm a new submitter to this channel...",Normal
780,780,Thank God the CB is over for Eid,Normal
...,...,...,...
23945,23945,"I am failing school, I just got fired from my ...",Suicidal
9955,9955,I feel like I am a prisoner to strong mood swi...,Depression
49605,49605,Strangest symptoms medicine helps with I reali...,Bipolar
18532,18532,I am going into a new school for 12th grade th...,Depression


In [13]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 5304 entries, 22261 to 23011
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5304 non-null   int64 
 1   statement   5263 non-null   object
 2   status      5304 non-null   object
dtypes: int64(1), object(2)
memory usage: 165.8+ KB


,Unnamed: 0
count,5304.000000
mean,26680.773379
std,15323.596857
min,4.000000
25%,13505.750000
50%,27083.500000
75%,39859.250000
max,53022.000000


In [14]:
df[df.isna().any(axis=1)]

,Unnamed: 0,statement,status
50354,50354,NaN,Bipolar
51209,51209,NaN,Personality disorder
52637,52637,NaN,Anxiety
51220,51220,NaN,Personality disorder
50571,50571,NaN,Bipolar
52627,52627,NaN,Anxiety
50022,50022,NaN,Bipolar
49384,49384,NaN,Stress
48460,48460,NaN,Stress
50768,50768,NaN,Personality disorder


In [15]:
df = df.dropna()
df.info()
df.describe()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 5263 entries, 22261 to 23011
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5263 non-null   int64 
 1   statement   5263 non-null   object
 2   status      5263 non-null   object
dtypes: int64(1), object(2)
memory usage: 164.5+ KB


(5263, 3)

In [16]:
df = df[["statement", "status"]]
df

,statement,status
22261,Just as the the title says. I feel like one is...,Depression
41400,a blackened sky encroached tugging behind it m...,Depression
20065,"It gives you insomnia, which in turn makes you...",Depression
30036,"Hello all, I'm a new submitter to this channel...",Normal
780,Thank God the CB is over for Eid,Normal
...,...,...
23945,"I am failing school, I just got fired from my ...",Suicidal
9955,I feel like I am a prisoner to strong mood swi...,Depression
49605,Strangest symptoms medicine helps with I reali...,Bipolar
18532,I am going into a new school for 12th grade th...,Depression


In [17]:
df.shape

(5263, 2)

In [18]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

CUDA available: True
GPU name: NVIDIA GeForce RTX 4060


In [19]:
df['status'].value_counts()

status
Normal                  1648
Depression              1603
Suicidal                 952
Anxiety                  377
Stress                   289
Bipolar                  278
Personality disorder     116
Name: count, dtype: int64

In [42]:
# possible_label = df.status.unique()
# dict_label = {}
# for index,possible_label in enumerate(possible_label):
#     dict_label[possible_label] = index
# dict_label

Preprocess

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_tags(string):
    result = re.sub(r'<.*?>', '', string)  
    result = re.sub(r'https?://\S+', '', result)  
    result = re.sub(r'[^\w\s]', ' ', result) 
    result = result.lower()
    return result

df['statement'] = df['statement'].apply(remove_tags)

stop_words = set(stopwords.words('english'))
df['statement'] = df['statement'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prig1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,statement,status
22261,title says feel like one hyper alot attention ...,Depression
41400,blackened sky encroached tugging behind depres...,Depression
20065,gives insomnia turn makes depression worse day...,Depression
30036,hello new submitter channel went nights ago ho...,Normal
780,thank god cb eid,Normal
...,...,...
23945,failing school got fired job walmart making bu...,Suicidal
9955,feel like prisoner strong mood swings way neve...,Depression
49605,strangest symptoms medicine helps realized foo...,Bipolar
18532,going new school 12th grade fall terrified get...,Depression


In [22]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['statement'] = df.statement.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prig1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
df

,statement,status
22261,title say feel like one hyper alot attention w...,Depression
41400,blackened sky encroached tugging behind depres...,Depression
20065,give insomnia turn make depression worse day m...,Depression
30036,hello new submitter channel went night ago hon...,Normal
780,thank god cb eid,Normal
...,...,...
23945,failing school got fired job walmart making bu...,Suicidal
9955,feel like prisoner strong mood swing way never...,Depression
49605,strangest symptom medicine help realized food ...,Bipolar
18532,going new school 12th grade fall terrified get...,Depression


train-test

In [24]:
reviews = df['statement'].values
labels = df['status'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [25]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [26]:
unique_labels = set(train_labels)
word_counts = {l: defaultdict(lambda: 0) for l in unique_labels}

In [27]:
print(set(train_labels))

{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)}


In [29]:
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()
X = X.toarray()
word_counts = {}
for l in range(7):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

In [ ]:
from collections import defaultdict

word_counts = defaultdict(lambda: defaultdict(int))

for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]


In [31]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

In [32]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data

In [ ]:
import math
from collections import defaultdict

def group_by_label(x, y, labels):
    grouped_data = defaultdict(list)
    for i, label in enumerate(y):
        grouped_data[label].append(x[i])
    return grouped_data

def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    
    grouped_data = group_by_label(x, y, labels)

    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)

    return n_label_items, log_label_priors

In [35]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

Evaluation

In [ ]:
labels = [0,1,2,3,4,5,6]
n_label_items, log_label_priors = fit(train_sentences, train_labels, labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)


from sklearn.metrics import classification_report


inv_dict_label = {v: k for k, v in dict_label.items()}
pred_labels = [inv_dict_label[p] for p in pred]
true_labels = [inv_dict_label[t] for t in test_labels]


print(labels)
print(train_labels)
print(classification_report(true_labels, pred_labels))

[0, 1, 2, 3, 4, 5, 6]
[3 3 5 ... 3 3 6]
                      precision    recall  f1-score   support

             Anxiety       0.94      0.07      0.14       412
             Bipolar       0.00      0.00      0.00        29
          Depression       1.00      0.02      0.04        94
              Normal       0.00      0.00      0.00        70
Personality disorder       0.60      0.01      0.02       238
              Stress       0.00      0.00      0.00        72
            Suicidal       0.31      1.00      0.48       401

            accuracy                           0.33      1316
           macro avg       0.41      0.16      0.10      1316
        weighted avg       0.57      0.33      0.19      1316



c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Save result for compare with bert

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd


inv_dict_label = {v: k for k, v in dict_label.items()}
pred_labels = [inv_dict_label[p] for p in pred]
true_labels = [inv_dict_label[t] for t in test_labels]


report_dict = classification_report(true_labels, pred_labels, output_dict=True)

df_report = pd.DataFrame(report_dict).transpose()

latex_table = df_report.to_latex(r"results\NB_classification_report.tex", float_format="%.2f")

c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\prig1\miniconda3\envs\IRtest\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
